In [ ]:
from multiprocessing import Pool
from tqdm.auto import tqdm
import cProfile, pstats
import numpy as np
from numba import njit, prange
%load_ext snakeviz

In [ ]:
#test pool with gen
items = ['a','b','c']
vals = (['a',[100,101,102]],['b',[200,201,202,203]],['c',[300,301]]) # list-based
j = iter(range(len(vals)))
k={}


In [ ]:
def test(item):
    kk = next(j)
    k[kk] = item
    for val in enumerate(item[1]):
        print(f'\n{item[0]}[{i}] has {val}')

In [ ]:
@njit(parallel=True)
def test_nb(item):
    for val in prange(item.shape[0]):
        print(f'\n {val}')

In [ ]:
def dtest(items):
    print(f'\n[{items[0]}] has {items[1]}')

In [ ]:
vs= {k:v for k,v in vals}
v_ks = {i:k for i,k in enumerate(vs.keys())}
val_count = sum( len(v) for v in list(vs.values()))
val_count

In [ ]:
ixn_keys = [[(ik,nk) for nk,_ in enumerate(vs[ v_ks[ik] ])] for ik in tuple(v_ks.keys())]
dummy_dict = {}
for inkl, nl in zip( ixn_keys,vs.values() ):
    print(f'associate members of {nl} to {inkl}')
    for ink,n in zip(inkl,nl):
        print(f'{ink} <-- {n}')
        dummy_dict[ink] = n #dict-based

In [ ]:
(a,b), c = list(dummy_dict.items())[0]
a,b,c

In [ ]:
def test_():
    pool = Pool()
    with pool:
        tuple( tqdm( pool.imap( test_nb, vals ), total=3) )

In [ ]:
def dtest_():
    pool = Pool()
    with pool:
        tuple( tqdm( pool.imap( dtest, dummy_dict.items() ), total=3) )

In [ ]:
def profile_f(f):
    profiler = cProfile.Profile()
    profiler.enable()
    f()
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('tottime')
    return stats

In [ ]:
# lt = profile_f(test_).print_stats()
# dt = profile_f(dtest_).print_stats()


In [ ]:
%snakeviz dtest_()